## Import stuff

In [232]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import torch 
import torch.jit
import torch.nn as nn 
import torchvision
from torchvision import transforms, datasets
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn import datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.impute import KNNImputer
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import ClusterCentroids
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

print(tf.__version__)

from tensorflow.python.client import device_lib
print("tensorflow doesn't detects GPU: ")
print(device_lib.list_local_devices() )
print(tf.test.gpu_device_name())
print(tf.config.list_physical_devices())

print("torch detects GPU: " , torch.cuda.is_available())

oversampler = RandomOverSampler(random_state=0)
undersampler = ClusterCentroids(random_state=0)


def get_score(df_true, df_submission):
    for i in range(df_true.shape[1]):
        print("score for col ",i,": " , metrics.roc_auc_score(df_true[:,i], df_submission[:,i]))


2.3.0
tensorflow doesn't detects GPU: 
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2963619102495831283
]

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
torch detects GPU:  True


## Load training data from csv


In [233]:
df = pd.read_csv("train_features_imputed_normalized.csv")
labels_df = pd.read_csv("train_labels.csv")

X = np.array(df.iloc[:, 1:].values,  dtype=float)
Y = np.array(labels_df.iloc[:, 1:].values,  dtype=float)

X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=0)

print("X shape: ", X.shape)
print("Y shape: ", Y.shape)



X shape:  (18995, 240)
Y shape:  (18995, 15)


## The function that trains and rates models

In [234]:
def find_best_model(col, X_train, Y_train, X_valid, Y_valid, X_test, X, Y):
    models = []

    Y_train = Y_train[:,col]
    Y_valid = Y_valid[:,col]
    # print(X_train.shape)
    # X_train = PolynomialFeatures(degree=2).fit_transform(X_train)
    # print(X_train.shape)
    # X_valid = PolynomialFeatures(degree=2).fit_transform(X_valid)
    # X_train = SelectKBest(chi2, k=35).fit_transform(X_train+0.01, Y_train)
    # X_valid = SelectKBest(chi2, k=35).fit_transform(X_valid+0.01, Y_valid)
    # X_train, Y_train = oversampler.fit_resample(X_train, Y_train)
    # X_train, Y_train = undersampler.fit_resample(X_train, Y_train)

    #task 1 and 2
    if(col<11):    
        print("training 64+64 relu for col ", col)
        model = tf.keras.Sequential([
            layers.Dense(64, activation=tf.keras.activations.relu),
            layers.Dense(64, activation=tf.keras.activations.relu),
            layers.Dense(1, activation='sigmoid')
        ])
        model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.01))
        model.fit(X_train, Y_train, epochs=15, verbose=False)
        models.append(model)

        print("training ridge",1," for col ", col)
        ridge = Ridge()
        ridge.fit(X_train, Y_train)
        models.append(ridge)

        # print("training GradientBoostingClassifier for col ", col)
        # model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0)
        # model.fit(X_train, Y_train)
        # models.append(model)
        
        # print("training ExtraTrees for col ", col)
        # model = ExtraTreesClassifier(n_estimators=100, random_state=0)
        # model.fit(X_train, Y_train)
        # models.append(model)

        # print("training rfc for col ", col)
        # rfc = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=10)
        # rfc.fit(X_train, Y_train)
        # models.append(rfc)

        # print("training decision tree for col ", col)
        # model = DecisionTreeClassifier(max_depth=5)
        # model.fit(X_train, Y_train)
        # models.append(model)
                        
        # print("training Naive Bayes for col ", col)
        # model = GaussianNB()
        # model.fit(X_train, Y_train)
        # models.append(model)

        # print("training adaboost for col ", col)
        # adaboost = AdaBoostClassifier(n_estimators=100, random_state=10)
        # adaboost.fit(X_train, Y_train)
        # models.append(adaboost)
                                        
        # print("training QDA for col ", col)
        # model = QuadraticDiscriminantAnalysis()
        # model.fit(X_train, Y_train)
        # models.append(model)
        
        # print("training lr for col ", col)
        # lr = LogisticRegression(max_iter=1000)
        # lr.fit(X_train, Y_train)
        # models.append(lr)

        # print("training kernel ridge for col ", col)
        # kernel_ridge = KernelRidge(alpha=1.0, degree=2)
        # kernel_ridge.fit(X_train, Y_train)
        # models.append(kernel_ridge)

        # print("training poly kernel SVM for col ", col)
        # svm_clf = SVC(kernel='poly', C=1.0, degree=2)
        # svm_clf.fit(X_train, Y_train)
        # models.append(svm_clf)

        # print("training rbf kernel SVM for col ", col)
        # rbf = svm.SVC(kernel="rbf",probability=True)
        # rbf.fit(X_train, Y_train)
        # models.append(rbf)

        # print("training poly kernel SVM for col ", col)
        # degree = 2
        # l2_coef = 0.01
        # model2 = make_pipeline(PolynomialFeatures(degree),Ridge(alpha=l2_coef, fit_intercept=False, solver="svd"))
        # model2.fit(X_train, Y_train)



        # this need too much ram
        # print("training Gaussian Process for col ", col)
        # gp = GaussianProcessClassifier()#1.0 * RBF(1.0)
        # gp.fit(X_train, Y_train)
        # models.append(gp)

    # task 2
    if(col==10):
        print("training ridge for col ", col)
        ridge = Ridge(alpha=1.0)
        ridge.fit(X_train, Y_train)
        models.append(ridge)
    
    #task 3
    if(col>=11):
        print("training 64+64 relu for col ", col)
        nn = tf.keras.Sequential([
        tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=1)
        ])
        nn.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.01))
        nn.fit(X_train, Y_train, epochs=30, verbose=False)
        models.append(nn)

    # if(col==13):
    #     print("training ridge for col ", col)
    #     ridge = Ridge(alpha=1.0)
    #     ridge.fit(X_train, Y_train)
    #     models.append(ridge)



    
    print("done training all models for col ", col)

    best_score = 0
    best_model = None
    probabilistic_models = ["SVC", "RandomForestClassifier", "GradientBoostingClassifier", "ExtraTreesClassifier", "RandomForestRegressor", "DecisionTreeClassifier", "GaussianNB", "AdaBoostClassifier", "QuadraticDiscriminantAnalysis", "GaussianProcessClassifier"]
    for model in models:
        model_type = str(type(model)).split(".")[-1].split("'")[0]
        # print("testing model ", model_type)
        #if the model is probabilistic
        if model_type in probabilistic_models:
            pred = model.predict_proba(X_valid)[:,1]
        else:
            pred = model.predict(X_valid)
        if(col<11):  
            np.clip(pred, 0.0, 1.0, out=pred)
            # print(pred.min())
            # print(pred.max())
        # print("predictions shape: ", pred.shape)
        if(col>=11):
            score = 0.5 + 0.5 * np.maximum(0, metrics.r2_score(Y_valid, pred))
        else:
            score = metrics.roc_auc_score(Y_valid, pred)
        print("score for",type(model),": " , score)
        if score > best_score:
            best_score = score
            best_model = model

    print("best model: ", type(best_model))

    #doesn't seem to work
    # if str(type(best_model)).count("Sequential") > 0:
    #     best_model.fit(X,Y[:,col], epochs=15, verbose=False)
    # else:
    #     best_model.fit(X, Y[:,col])

    predictions = best_model.predict(X_test)
    # print("predictions shape: ", predictions.shape)

    return predictions

## Writing results

In [235]:

# X_test = pd.read_csv("test_features_normalized.csv")
# X_test = np.array(X_test.iloc[:, 2:].values,  dtype=float).reshape(151968//12,35*12)
X_test = pd.read_csv("test_features_imputed_normalized.csv")
X_test = np.array(X_test.iloc[:, 1:].values,  dtype=float)

data = []

for i in range(0,15):
# for i in range(10,11):
# for i in range(11,15):
# for i in range(13,14):
    # X = SelectKBest(chi2, k=10).fit_transform(X[:,:35]+0.01, Y[:,i])
    # pred = find_best_model(i, X[:int(X.shape[0]*0.8),:35], Y[:int(Y.shape[0]*0.8),:35], X[int(X.shape[0]*0.8):,:35], Y[int(Y.shape[0]*0.8):,:35], X_te st)

    # pred = find_best_model(i, X, Y, X[int(X.shape[0]*0.8):], Y[int(Y.shape[0]*0.8):], X_test,X,Y)
    pred = find_best_model(i, X[:int(X.shape[0]*0.8)], Y[:int(Y.shape[0]*0.8)], X[int(X.shape[0]*0.8):], Y[int(Y.shape[0]*0.8):], X_test,X,Y)
    # pred = find_best_model(i, X[int(X.shape[0]*0.2):], Y[int(Y.shape[0]*0.2):], X[:int(X.shape[0]*0.2)], Y[:int(Y.shape[0]*0.2)], X_test,X,Y)
    # print("pred shape: ", pred.shape)
    pred = pred.reshape(pred.shape[0],1)
    data.append(pred)





training 64+64 swish for col  0
training 64+64 relu for col  0
training ridge 1  for col  0
done training all models for col  0
score for <class 'tensorflow.python.keras.engine.sequential.Sequential'> :  0.9285835187155431
score for <class 'tensorflow.python.keras.engine.sequential.Sequential'> :  0.9301552374504304
score for <class 'sklearn.linear_model._ridge.Ridge'> :  0.9224130545921821
best model:  <class 'tensorflow.python.keras.engine.sequential.Sequential'>
training 64+64 swish for col  1
training 64+64 relu for col  1
training ridge 1  for col  1
done training all models for col  1
score for <class 'tensorflow.python.keras.engine.sequential.Sequential'> :  0.80193823313783
score for <class 'tensorflow.python.keras.engine.sequential.Sequential'> :  0.799167073965461
score for <class 'sklearn.linear_model._ridge.Ridge'> :  0.7853779732811991
best model:  <class 'tensorflow.python.keras.engine.sequential.Sequential'>
training 64+64 swish for col  2
training 64+64 relu for col  2


KeyboardInterrupt: 

In [ ]:
np_array = np.array(data).transpose()[0]
# np_array[:,13] = (np_array[:,13]*73)+27
col_names = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2', 'LABEL_Bilirubin_direct', 'LABEL_EtCO2', 'LABEL_Sepsis', 'LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']
print(len(col_names))
new_df = pd.DataFrame(np_array, columns=col_names)

output_csv = pd.read_csv("sample.csv")
output_csv.update(new_df)
output_csv.to_csv('submit.zip', index=False, float_format='%.3f', compression='zip')
output_csv.to_csv('submit.csv', index=False, float_format='%.3f')


15
